In [1]:
LOADABLE_ROWS = 1
FILE_NAME = 'ec_instance.txt'

In [2]:
class Matrix:
    def __init__(self, data, rows, cols):
        self.data = data
        self.rows = rows
        self.cols = cols

In [3]:
def count_comment_lines(filename):
    with open(filename, 'r') as f:
        return sum(1 for line in f if line.startswith(";;;"))

In [4]:
def count_total_lines(filename):
    with open(filename, 'r') as f:
        return sum(1 for line in f)

In [5]:
def detect_columns(filename, offset):
    with open(filename, 'r') as f:
        for _ in range(offset):
            next(f)  # skip lines
        line = next(f).strip().split()
        return len(line) - 1  # excluding the trailing "-"

In [6]:
def read_file(filename, rows, cols, offset):
    with open(filename, 'r') as f:
        for _ in range(offset):
            next(f)  # skip lines

        data = []


        for k in range(rows):

            line = next(f, None)
            if line is None:
                break
            numbers = list(map(int, line.strip().split()[:-1]))  # excluding the trailing "-"
            if len(numbers) != cols:
                break

            data.extend(numbers)

        matrix_rows = len(data) // cols
        return Matrix(data, matrix_rows, cols)

In [7]:
def intersect(row1, row2):
    """Returns True if two rows have a common '1', else False"""
    return any(a == b == 1 for a, b in zip(row1, row2))

def union(row1, row2):
    """Return the union of two rows (bitwise OR)"""
    return [a | b for a, b in zip(row1, row2)]

In [151]:
def EC(A, B, COV, offset, FILE_NAME, LOADABLE_ROWS, explored=set()):
    N = A.rows
    M = A.cols

    comment_lines = count_comment_lines(FILE_NAME)


    for i in range(N):
        row_data = A.data[i * M:i * M + M] 


        # To keep the corrispondence between the rows of A
        # and the columns of B, 
        # the column B[:,i+offset] is put to 0
        # and the row B[i+offset,:] is put to 0
        if sum(row_data) == 0:
            for t in range(N):
                B[t][i+offset] = 0
                B[i+offset][t] = 0

            # Print B
            # print("B got a zero row/column")
            # print(B)
            continue

        if sum(row_data) == M:
            for t in range(N):
                B[t][i+offset] = 0
                B[i+offset][t] = 0
            # print("B got a zero row/column")
            # print(B)
            COV.add((offset + i,))
            continue

        # Iterate over previous rows of A in chunks
        reading_offset = 0
        print("start while outside")
        while reading_offset < i + offset:
            print("start while inside")
            old_A = read_file(FILE_NAME, LOADABLE_ROWS, M, reading_offset+comment_lines)
            # Logically, the for would be from 0 to i+offset excluded,
            # so we need to ignore all j greater than i+offset
            for j in range(min(old_A.rows, i + offset - reading_offset)):
                # if j + reading_offset >= i + offset:
                #     print("breaking since j + reading_offset >= i + offset")
                #     break
                
                # if old_A.data[j * M:j * M + M] is all zeros or all ones
                # then we can skip this row
                if sum(old_A.data[j * M:j * M + M]) == 0 or sum(old_A.data[j * M:j * M + M]) == M:
                    continue

                if intersect(old_A.data[j * M:j * M + M], row_data):
                    B[j + reading_offset][i + offset] = 0
                else:
                    I = {offset + i, j + reading_offset}
                    U = union(old_A.data[j * M:j * M + M], row_data)
                    if sum(U) == M:
                        COV.add(tuple(sorted(I)))
                        B[j + reading_offset][i + offset] = 0
                    else:
                        B[j + reading_offset][i + offset] = 1
                        inter = [k for k in range(j + reading_offset) if B[k][i + offset] and B[k][j + reading_offset]]
                        if inter:
                            explore(I, U, inter, COV, B, offset, old_A.cols, FILE_NAME, LOADABLE_ROWS, explored)

            reading_offset += LOADABLE_ROWS
            print("end while inside")
        print("end while outside")

    return COV


def explore(I, U, inter, COV, B, offset, M, FILE_NAME, LOADABLE_ROWS, explored=set()):
    # print("Exploring")
    # print("I: ", I)
    if I in explored:
        print("Already explored")
        return
    explored.add(tuple(sorted(I)))
    comment_lines = count_comment_lines(FILE_NAME)
    for k in inter:
        i_temp = I.union({k})
        
        # Get the data corresponding to row 'k' from the file
        chunk_index = (k // LOADABLE_ROWS) * LOADABLE_ROWS
        chunk = read_file(FILE_NAME, LOADABLE_ROWS, M, chunk_index + comment_lines)
        row_data = chunk.data[(k % LOADABLE_ROWS) * M:(k % LOADABLE_ROWS + 1) * M]
        
        u_temp = union(U, row_data)
        if sum(u_temp) == M:
            COV.add(tuple(sorted(i_temp)))
        else:
            inter_temp = [l for l in inter if l < k and B[l][k]]
            if inter_temp:
                explore(i_temp, u_temp, inter_temp, COV, B, offset, M, FILE_NAME, LOADABLE_ROWS, explored)


In [106]:
# Testing how set changes when passed to a function and updated
def test_set(s):
    s.update({1, 2, 3, 6, 7})

s = set({1,2,3,4,5})
test_set(s)
s

{1, 2, 3, 4, 5, 6, 7}

In [143]:
def incremental_process(A, B, COV, offset, FILE_NAME, LOADABLE_ROWS, explored=set()):
    
    # Find exact covers within the new chunk
    chunk_COV = EC(A, B, set(), offset, FILE_NAME, LOADABLE_ROWS, explored)
    # print("Chunk COV:", chunk_COV)
    COV.update(chunk_COV)

    return COV

In [126]:
# Creating a function that takes a filename and a number of loadable rows
# and solves the exact cover problem incrementally:

def incremental_exact_cover(filename, loadable_rows, verbose=False):

    # Counting the comments
    comment_lines = count_comment_lines(filename)
    
    # Detecting the number of columns
    n_columns = detect_columns(filename, comment_lines)
    print(f"Total columns: {n_columns}")

    # Counting the total number of rows
    total_rows = count_total_lines(filename) - comment_lines
    print(f"Total rows: {total_rows}")

    # Initializing the matrix B
    B = [[0] * total_rows for _ in range(total_rows)]

    # Initializing the set of visited nodes
    explored = set()

    # Initializing the set of partitions (solutions)
    COV = set()

    # Initializing the offset
    offset = 0

    while True:
        matrix = read_file(filename, loadable_rows, n_columns, offset+comment_lines)
        if not matrix.data:
            break

        if verbose:
            print("Portion of A read: from row", offset+1, "to row", offset+matrix.rows)
        # for i in range(matrix.rows):
        #     print(matrix.data[i * matrix.cols:i * matrix.cols + matrix.cols])
        # print("Found complete sets:", matrix.ones)

        incremental_process(matrix, B, COV, offset, filename, loadable_rows, explored)

        if verbose:
            print("Explored:", len(explored))

            print("B:")
            for row in B:
                print(row)

            print("COV:")
            print(COV)

        offset += matrix.rows

    print("Explored:", len(explored))

    print("Solutions:", len(COV))

    print("B:")
    for row in B:
        print(row)

    print("COV:")
    print(COV)

    return COV

When `loadable_rows` is large the number of times the while loop is executed as a whole is the same
as when the `loadable_rows` is small but the number of iterations of the while loop is smaller,
but the time it takes for each iteration is much longer.

When `loadable_rows` is small, the number of iterations of the while loop is larger but the time it takes for
each iteration is much shorter.

In [156]:
# Testing the resolution on a small instance
cov = incremental_exact_cover("ec_instance.txt", 11)

print("Solutions:")
for solution in cov:
    sets = [f"S_{i+1}" for i in solution]
    print(", ".join(sets))

Total columns: 10
Total rows: 9
start while outside
end while outside
start while outside
start while inside
end while inside
end while outside
start while outside
start while inside
end while inside
end while outside
start while outside
start while inside
end while inside
end while outside
start while outside
start while inside
end while inside
end while outside
start while outside
start while inside
end while inside
end while outside
start while outside
start while inside
end while inside
end while outside
start while outside
start while inside
end while inside
end while outside
Explored: 4
Solutions: 4
B:
[0, 1, 1, 1, 0, 1, 0, 0, 0]
[0, 0, 0, 1, 0, 0, 1, 1, 0]
[0, 0, 0, 0, 0, 1, 0, 1, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 1]
[0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 1, 0, 1]
[0, 0, 0, 0, 0, 0, 0, 1, 1]
[0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0]
COV:
{(5, 6, 8), (0, 1, 3), (1, 6, 7), (0, 2, 5)}
Solutions:
S_6, S_7, S_9
S_1, S_2, S_4
S_2, S_7, S_8
S_1, S_3, S_6


In [ ]:
# Input sudoku example:

# 5, 3, 0, 0, 7, 0, 0, 0, 0,
# 6, 0, 0, 1, 9, 5, 0, 0, 0,
# 0, 9, 8, 0, 0, 0, 0, 6, 0,
# 8, 0, 0, 0, 6, 0, 0, 0, 3,
# 4, 0, 0, 8, 0, 3, 0, 0, 1,
# 7, 0, 0, 0, 2, 0, 0, 0, 6,
# 0, 6, 0, 0, 0, 0, 2, 8, 0,
# 0, 0, 0, 4, 1, 9, 0, 0, 5,
# 0, 0, 0, 0, 8, 0, 0, 7, 9,

In [66]:
def read_sudoku_from_file(filename):
    with open(filename, 'r') as file:
        lines = file.readlines()
    
    sudoku = []
    for line in lines:
        row = [int(num) for num in line.strip().split(",") if num]
        sudoku.extend(row)

    return sudoku

# Test
sudoku_sdk_filename = "sudoku2.sdk.txt"
sudoku = read_sudoku_from_file(sudoku_sdk_filename)
for i in range(9):
    print(sudoku[i*9:i*9+9])


[5, 3, 4, 6, 7, 8, 9, 1, 2]
[6, 7, 2, 1, 9, 5, 3, 4, 8]
[1, 9, 8, 3, 4, 2, 5, 6, 7]
[8, 5, 9, 7, 6, 1, 4, 2, 3]
[4, 2, 6, 8, 5, 3, 7, 9, 1]
[7, 1, 3, 9, 2, 4, 8, 5, 6]
[9, 6, 1, 5, 3, 7, 2, 8, 4]
[2, 8, 7, 4, 1, 9, 6, 3, 5]
[3, 4, 5, 2, 8, 6, 1, 7, 0]


In [67]:
def sudoku_to_exact_cover_9_x_9(sudoku):
    N = 9
    constraints = 4  # Cell, Row, Column, Box
    cover_matrix = [[0] * (N * N * constraints) for _ in range(N * N * N)]

    for r in range(N):
        for c in range(N):
            for n in range(1, N + 1):
                # Calculate row index for cover_matrix
                idx = (r * N + c) * N + n - 1
                
                # Cell constraint
                cover_matrix[idx][r * N + c] = 1

                # Row constraint
                cover_matrix[idx][N * N + r * N + n - 1] = 1
                
                # Column constraint
                cover_matrix[idx][2 * N * N + c * N + n - 1] = 1
                
                # Box constraint
                box_row = r // 3
                box_col = c // 3
                box_num = box_row * 3 + box_col
                cover_matrix[idx][3 * N * N + box_num * N + n - 1] = 1
                
    # Prune rows that conflict with given Sudoku puzzle
    rows_to_remove = []
    for r in range(N):
        for c in range(N):
            num = sudoku[r * N + c]
            if num:  # If cell is filled in Sudoku
                start_idx = (r * N + c) * N
                # Remove all rows for this cell, except the one corresponding to 'num'
                for i in range(1, N + 1):
                    if i != num:
                        rows_to_remove.append(start_idx + i - 1)

    # Remove rows in reverse to avoid index issues
    for idx in sorted(rows_to_remove, reverse=True):
        del cover_matrix[idx]
                
    return cover_matrix

In [68]:
def sudoku_to_exact_cover_4_x_4(sudoku):
    N = 4  # This represents the size of the Sudoku (4x4 in this case)
    constraints = 4  # Cell, Row, Column, Box
    cover_matrix = [[0] * (N * N * constraints) for _ in range(N * N * N)]

    for r in range(N):
        for c in range(N):
            for n in range(1, N + 1):
                # Calculate row index for cover_matrix
                idx = (r * N + c) * N + n - 1
                
                # Cell constraint
                cover_matrix[idx][r * N + c] = 1

                # Row constraint
                cover_matrix[idx][N * N + r * N + n - 1] = 1
                
                # Column constraint
                cover_matrix[idx][2 * N * N + c * N + n - 1] = 1
                
                # Box constraint
                box_row = r // 2  # This has changed from 3 to 2 for 4x4 Sudoku
                box_col = c // 2  # This has changed from 3 to 2 for 4x4 Sudoku
                box_num = box_row * 2 + box_col  # This has changed from 3 to 2 for 4x4 Sudoku
                cover_matrix[idx][3 * N * N + box_num * N + n - 1] = 1
                
    # Prune rows that conflict with given Sudoku puzzle
    # by setting them to all zeros

    rows_to_remove = []
    for r in range(N):
        for c in range(N):
            num = sudoku[r * N + c]
            if num:
                start_idx = (r * N + c) * N
                for i in range(1, N + 1):
                    if i != num:
                        rows_to_remove.append(start_idx + i - 1)

    # Remove rows in reverse to avoid index issues
    for idx in sorted(rows_to_remove, reverse=True):
        # Setting the row to all zeros
        cover_matrix[idx] = [0] * (N * N * constraints)
                
    return cover_matrix


In [69]:
def exact_cover_solution_to_sudoku_4_x_4(partition):
    N = 4
    solution = [[0] * N for _ in range(N)]
    
    for idx in partition:
        r, c, n = idx // (N * N), (idx // N) % N, (idx % N) + 1
        solution[r][c] = n
        
    return solution

In [70]:
# Function that takes in input a sudoku.sdk filename and in output a sudoku.exc filename
# the default comment is a timestamp
import datetime

def sdk_to_exc(sdk_filename, exc_filename, comment=datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S"), size=9):
    sudoku = read_sudoku_from_file(sdk_filename)
    if size == 4:
        cover_matrix = sudoku_to_exact_cover_4_x_4(sudoku)
    else:
        cover_matrix = sudoku_to_exact_cover_9_x_9(sudoku)
    with open(exc_filename, "w") as f:
        # for each line, append ";;; " at the beginning
        for line in comment.split("\n"):
            f.write(";;; " + line + "\n")
        for row in cover_matrix:
            f.write(" ".join(map(str, row)) + " -\n")

In [108]:
# Would take a lot of time to be solved!
sdk_to_exc("sudoku1.sdk.txt", "sudoku1.exc.txt")
sdk_to_exc("sudoku2.sdk.txt", "sudoku2.exc.txt")

In [71]:
sdk_to_exc("sudoku_easy_4x4.sdk.txt", "sudoku_easy_4x4.exc.txt", size=4)
sdk_to_exc("sudoku_medium_4x4.sdk.txt", "sudoku_medium_4x4.exc.txt", size=4)
sdk_to_exc("sudoku_hard_4x4.sdk.txt", "sudoku_hard_4x4.exc.txt", size=4)


When `loadable_rows` is small the number of times 

In [154]:
# This easy sudoku already takes around 2 minutes to be solved
# 1, 2, 0, 4,
# 4, 3, 2, 0,
# 3, 1, 0, 2,
# 2, 4, 0, 0,

sol_sudoku_easy_4x4 = incremental_exact_cover("sudoku_easy_4x4.exc.txt", loadable_rows=1)
print("Solutions:")
print(sol_sudoku_easy_4x4)

Total columns: 64
Total rows: 64
start while outside
end while outside
start while outside
start while inside
end while inside
start while inside
end while inside
start while inside
end while inside
start while inside
end while inside
start while inside
end while inside
end while outside
start while outside
start while inside
end while inside
start while inside
end while inside
start while inside
end while inside
start while inside
end while inside
start while inside
end while inside
start while inside
end while inside
start while inside
end while inside
start while inside
end while inside
end while outside
start while outside
start while inside
end while inside
start while inside
end while inside
start while inside
end while inside
start while inside
end while inside
start while inside
end while inside
start while inside
end while inside
start while inside
end while inside
start while inside
end while inside
start while inside
end while inside
end while outside
start while outside
sta

In [87]:
solved_sudoku_easy_4x4 = exact_cover_solution_to_sudoku_4_x_4(sol_sudoku_easy_4x4.pop())
print("Solved Sudoku:")
for row in solved_sudoku_easy_4x4:
    print(row)

Solved Sudoku:
[1, 2, 3, 4]
[4, 3, 2, 1]
[3, 1, 4, 2]
[2, 4, 1, 3]


In [ ]:
# This would take a lot more!

# sol_sudoku_medium_4x4 = incremental_exact_cover("sudoku_medium_4x4.exc.txt", 10)
# print("Solutions:")
# print(sol_sudoku_medium_4x4)

In [ ]:
# ... and even more!
# sol_sudoku_hard_4x4 = incremental_exact_cover("sudoku_hard_4x4.exc.txt", 10)
# print("Solutions:")
# print(sol_sudoku_hard_4x4)

In [74]:
import random

def generate_exact_cover(N, M, filename):
    if M > N:
        raise ValueError("M should be less than or equal to N for guaranteed solution.")
    
    # Step 1: Start with an empty NxM matrix
    matrix = [[0 for _ in range(M)] for _ in range(N)]
    
    # Step 2: Choose a solution
    selected_rows = random.sample(range(N), M)
    for i, row in enumerate(selected_rows):
        matrix[row][i] = 1

    # Step 3: Add noise to remaining rows
    for i in range(N):
        if i not in selected_rows:
            for j in range(M):
                matrix[i][j] = random.choice([0, 1])

    # Step 4: Write to file
    with open(filename, "w") as f:
        f.write(";;; Generated exact cover problem\n")
        f.write(";;; " + datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S") + "\n")
        f.write(";;; N = " + str(N) + "\n")
        f.write(";;; M = " + str(M) + "\n")
        f.write(";;; Solution: " + ", ".join(map(str, selected_rows)) + "\n")
        for row in matrix:
            f.write(" ".join(map(str, row)) + " -\n")
    
    return matrix

In [128]:
ec_1 = generate_exact_cover(20, 15, "generated_exact_cover.txt")
print("Generated exact cover problem:")
for row in ec_1:
    for num in row[:-1]:
        print(f"{num},", end=" ")
    print(row[-1], "-")

Generated exact cover problem:
0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0 -
0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0 -
0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0 -
1, 1, 0, 1, 1, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1 -
0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0 -
0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0 -
0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0 -
1, 1, 1, 0, 0, 0, 0, 1, 1, 1, 1, 0, 1, 1, 1 -
0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0 -
0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1 -
0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0 -
0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0 -
0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0 -
0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0 -
1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0 -
1, 1, 0, 0, 0, 1, 1, 1, 1, 1, 0, 0, 0, 1, 1 -
1, 1, 1, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1 -
0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0 -
0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0 -
0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0 -


In [139]:
# measuring the time to solve the generated problem
# with magic command %timeit
incremental_exact_cover(filename="generated_exact_cover.txt", loadable_rows=2, verbose=False)

Total columns: 15
Total rows: 20
Chunk COV: set()
Chunk COV: set()
Chunk COV: set()
Chunk COV: set()
Chunk COV: set()
Chunk COV: set()
Chunk COV: set()
Chunk COV: {(0, 1, 2, 4, 5, 13, 15)}
Chunk COV: set()
Chunk COV: {(3, 6, 11, 12, 13, 18, 19), (0, 4, 5, 6, 10, 11, 12, 16, 19), (0, 1, 2, 4, 5, 6, 9, 10, 11, 12, 13, 14, 17, 18, 19), (0, 1, 2, 6, 7, 19), (1, 2, 5, 8, 9, 10, 11, 12, 13, 14, 18, 19)}
Explored: 17565
Solutions: 6
B:
[0, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
[0, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1]
[0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1]
[0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 1, 1]
[0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
[0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
[0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 1]
[0, 0, 0, 0, 0, 0,

{(0, 1, 2, 4, 5, 6, 9, 10, 11, 12, 13, 14, 17, 18, 19),
 (0, 1, 2, 4, 5, 13, 15),
 (0, 1, 2, 6, 7, 19),
 (0, 4, 5, 6, 10, 11, 12, 16, 19),
 (1, 2, 5, 8, 9, 10, 11, 12, 13, 14, 18, 19),
 (3, 6, 11, 12, 13, 18, 19)}

In [124]:
incremental_exact_cover("ec_manual.txt", 100)

Total columns: 8
Total rows: 10
Chunk COV: {(2, 4), (5, 7), (0, 2, 9), (0, 1, 2), (3,)}
Explored:  2
B:
[0, 1, 1, 0, 0, 0, 0, 0, 0, 1]
[0, 0, 1, 0, 0, 0, 0, 0, 1, 0]
[0, 0, 0, 0, 0, 1, 0, 0, 0, 1]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 1]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
COV:
{(2, 4), (0, 2, 9), (0, 1, 2), (3,), (5, 7)}


{(0, 1, 2), (0, 2, 9), (2, 4), (3,), (5, 7)}